In [1]:
# coding=utf-8

import sqlite3
import numpy as np
import pandas as pd
import re

In [2]:
if __name__ == "__main__":
    con = sqlite3.connect("C:/Users/SAMSUNG/Downloads/corpus.db")
    cur = con.cursor()

In [3]:
query = cur.execute("SELECT * From SEJONG_CORPUS")

In [4]:
cols = [column[0] for column in query.description]

In [5]:
df = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)

In [6]:
df

,pk,ko_text,thai_text,file_name,sent_count,cumsum_count
0,1,아이티의 임시 정부를 이끌게 될 대통령으로 13일 취임한 에르타 트루이요 여사는 남...,นางเออร์ตาทรูโยซึ่งเข้ารับตำแหน่งเมื่อวันที่ 1...,BA90A073.txt,1,1
1,2,트루이요는 34세의 나이에 대법원 12 명의 법관 중 최초의 여성 법관으로 임명됐으...,เมื่ออายุ 34 ปี Trujillo ได้รับแต่งตั้งเป็นผู้...,BA90A073.txt,1,1
2,3,아이티 여성의 법적 지위등 2권의 책을 저술한 트루이요는 임시 대통령직을 수락하면서...,Truiyo ผู้แต่งหนังสือสองเล่มรวมถึงสถานะทางกฎหม...,BA90A073.txt,1,1
3,4,그녀가 지금부터 해 내야 할 일은 지난 86년 뒤발리에의 독재 정권이 무너진 지 4...,สิ่งที่เธอต้องทำในตอนนี้ก็คือการจัดการเลือกตั้...,BA90A073.txt,1,3
4,5,이것은 너무나도 벅찬 임무라서 누구도 감히 떠맡으려하지 않았던 일이다.,นี่เป็นงานที่หนักหน่วงจนไม่มีใครกล้าเข้าครอบครอง,BA90A073.txt,1,3
...,...,...,...,...,...,...
497386,500198,왜 안 믿기나,ทำไมไม่เชื่อ,BREO0335.txt,1,3
497387,500199,자리에서 일어나 육손이형께로 다가선 주 반장은 코빼기가 서로 닿을락말락 얼굴을 바짝...,จูจูผู้ซึ่งลุกขึ้นจากที่นั่งและเข้าหาฮยองลูกชา...,BREO0335.txt,1,6
497388,500200,당장 이 자리를 빠져나가고 싶다는 충동이 일어 막 자리에서 일어나려던 찰나 내 눈앞...,มีการกระตุ้นให้ออกจากสถานที่นี้ในขณะนี้และเมื่...,BREO0335.txt,1,6
497389,500201,서로의 팔을 기마전 안장 겯듯 끼고는 용을 쓰며 버티는 형국이었다.,เขาสวมมังกรในขณะที่สวมแขนของกันและกันเหมือนอาน,BREO0335.txt,1,6


In [7]:
 df.rename(columns = {"ko_text": "text_ko", "thai_text": "text_th"}, inplace = True)

In [9]:
con.close()

In [10]:
def clean_up_text(x):
    return  x.apply(lambda x: re.sub(r'\n', '',x) 
                         ).apply(lambda x: re.sub(r'\t', '',x) 
                         ).apply(lambda x: re.sub(r'<note>(.*?)</note>', '',x) 
                         ).apply(lambda x: re.sub(r'<(.*?)>', '',x)
                         ).apply(lambda x: re.sub(r'>', '',x) 
                         ).apply(lambda x: re.sub(r'<', '',x) 
                         ).apply(lambda x: re.sub(r's/^[*]{2}/','',x) 
                         ).apply(lambda x: re.sub(r's/.$//','',x)
                         ).apply(lambda x: re.sub(' +', ' ',x) 
                         ).apply(lambda x: re.sub('&quot;', '',x) 
                         )

In [13]:
df.text_ko = clean_up_text(df.text_ko)
df.text_th = clean_up_text(df.text_th)

In [ ]:
output_directory = "C:/Users/SAMSUNG/Desktop/output"
first_lang='ko'
sec_lang='th'
print("output_directory:",output_directory)

for lang in [first_lang,sec_lang]:

    train = df.sample(frac=0.8,random_state=1234) #random state is a seed value
    test  = df.drop(train.index)
    print(train.shape , test.shape)

    for split in ['train','test']:
        output_file_path =output_directory+"/"+split+".tags."+first_lang+"-"+sec_lang+"."+lang
        col_name='text_'+lang
        print(output_file_path)
        
        df_split = train if split=='train' else test
    
        with open(output_file_path,"w+", encoding='utf-8') as f:
            df_split[[col_name]].apply(lambda x: f.write(x.values[0]+"\n"),axis=1)
            f.close()